In [1]:
from urllib import request

import tensorflow as tf

from io import StringIO
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import nltk
from __future__ import print_function
from keras.layers import Dense, Activation, SimpleRNN, LSTM
from keras.models import Sequential


2024-11-14 18:13:20.912814: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 18:13:20.913191: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 18:13:20.915486: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 18:13:20.922110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731600800.933190    4224 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731600800.93

In [2]:
print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)

TensorFlow version: 2.18.0
NumPy version: 1.21.2


In [3]:
foo = request.urlopen('https://www.gutenberg.org/cache/epub/74708/pg74708.txt').read()

In [4]:
str_read = foo.decode('utf-8-sig')
str_clean = str_read.replace('\\r', '').replace('\\n', '').replace("&amp;", "&").replace("—", "").replace("\\_", "").replace('"', "")
# Remove hexadecimal byte sequences
str_clean = re.sub(r'\\x[0-9A-Fa-f]{2}', '', str_clean)


# Define sep_chars and create index dicts
sep_chars = sorted(set(str_clean.lower())) 
char2index = {c: i for i, c in enumerate(sep_chars)}
index2char = {i: c for i, c in enumerate(sep_chars)}  # Reverse of char2index
nb_chars = len(char2index)

In [5]:
actual_start = 'When Eulenspiegel'
word_index = str_clean.find(actual_start)

In [6]:
if word_index != -1:
    str_clean = str_clean[word_index:]


In [7]:
print(sep_chars)

['\n', '\r', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ä', 'æ', 'è', 'é', 'ï', 'ö', 'ü', 'œ', '‘', '’', '“', '”', '•', '™']


In [8]:
char2index = dict((c, i) for i, c in enumerate(sep_chars))
index_chars = dict((i, c) for i, c in enumerate(sep_chars))
nb_chars = len(char2index)

In [9]:
print(char2index)

{'\n': 0, '\r': 1, ' ': 2, '!': 3, '#': 4, '$': 5, '%': 6, '&': 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56, 'à': 57, 'ä': 58, 'æ': 59, 'è': 60, 'é': 61, 'ï': 62, 'ö': 63, 'ü': 64, 'œ': 65, '‘': 66, '’': 67, '“': 68, '”': 69, '•': 70, '™': 71}


In [10]:
print(index_chars)

{0: '\n', 1: '\r', 2: ' ', 3: '!', 4: '#', 5: '$', 6: '%', 7: '&', 8: '(', 9: ')', 10: '*', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '?', 28: '[', 29: ']', 30: '_', 31: 'a', 32: 'b', 33: 'c', 34: 'd', 35: 'e', 36: 'f', 37: 'g', 38: 'h', 39: 'i', 40: 'j', 41: 'k', 42: 'l', 43: 'm', 44: 'n', 45: 'o', 46: 'p', 47: 'q', 48: 'r', 49: 's', 50: 't', 51: 'u', 52: 'v', 53: 'w', 54: 'x', 55: 'y', 56: 'z', 57: 'à', 58: 'ä', 59: 'æ', 60: 'è', 61: 'é', 62: 'ï', 63: 'ö', 64: 'ü', 65: 'œ', 66: '‘', 67: '’', 68: '“', 69: '”', 70: '•', 71: '™'}


In [11]:
print(str_clean)

When Eulenspiegel came to Magdeburgthe fame of his notorious pranks
having preceded himseveral of the best citizens asked him to give
them a sample of his buffoonery. He said he would do so, and promised
to fly from the roof of a house on the market place. The news spread
rapidly, and old and young hurried there to see him fly. For some time
Eulenspiegel stood on the low roof, moving his arms and acting as if he
were going to fly, then he burst out laughing and cried: “I thought I was
the only fool in the world, but I see that here are almost a whole city
full of them. If you had told me that you were going to fly, I should not
have believed you; and yet you believed me, a well-known fool. How should
I be able to fly? I am neither a goose nor any other kind of a bird. I
have no wings, and without wings and feathers nobody can fly. Now you
see, I told you a falsehood.”

He turned and left the roof, while the crowd separated, some laughing,
some swearing, but all agreeing, that though he

In [12]:
text_tokenized = nltk.word_tokenize(str_clean)
print(text_tokenized)

['When', 'Eulenspiegel', 'came', 'to', 'Magdeburgthe', 'fame', 'of', 'his', 'notorious', 'pranks', 'having', 'preceded', 'himseveral', 'of', 'the', 'best', 'citizens', 'asked', 'him', 'to', 'give', 'them', 'a', 'sample', 'of', 'his', 'buffoonery', '.', 'He', 'said', 'he', 'would', 'do', 'so', ',', 'and', 'promised', 'to', 'fly', 'from', 'the', 'roof', 'of', 'a', 'house', 'on', 'the', 'market', 'place', '.', 'The', 'news', 'spread', 'rapidly', ',', 'and', 'old', 'and', 'young', 'hurried', 'there', 'to', 'see', 'him', 'fly', '.', 'For', 'some', 'time', 'Eulenspiegel', 'stood', 'on', 'the', 'low', 'roof', ',', 'moving', 'his', 'arms', 'and', 'acting', 'as', 'if', 'he', 'were', 'going', 'to', 'fly', ',', 'then', 'he', 'burst', 'out', 'laughing', 'and', 'cried', ':', '“', 'I', 'thought', 'I', 'was', 'the', 'only', 'fool', 'in', 'the', 'world', ',', 'but', 'I', 'see', 'that', 'here', 'are', 'almost', 'a', 'whole', 'city', 'full', 'of', 'them', '.', 'If', 'you', 'had', 'told', 'me', 'that', '

In [13]:
str_clean = str_clean.lower()
sequence_of_chars = 70
step = 1
input_chars = []
label_chars = []
for i in range(0, len(sep_chars) - sequence_of_chars, step):
    input_chars.append(str_clean[i:i + sequence_of_chars])
    label_chars.append(str_clean[i + sequence_of_chars])

In [14]:
print(input_chars)

['when eulenspiegel came to magdeburgthe fame of his notorious pranks\r\nh', 'hen eulenspiegel came to magdeburgthe fame of his notorious pranks\r\nha']


In [15]:
print(label_chars)

['a', 'v']


In [16]:
X = np.zeros((len(input_chars), sequence_of_chars, nb_chars), dtype=np.float32)
y = np.zeros((len(input_chars), nb_chars), dtype=np.float32)

for i, input_char in enumerate(input_chars):
    print(f"Loop number: {i} input_characters {input_char}")
    for j, ch in enumerate(input_char):
        print(f"Loop number: {j} Fetching char {ch}")
        print(f"index for char {char2index[ch]} adding to X...")
        X[i, j, char2index[ch]] = 1
    print(f"Adding to labels: {char2index[label_chars[i]]}")
    y[i, char2index[label_chars[i]]] = 1



Loop number: 0 input_characters when eulenspiegel came to magdeburgthe fame of his notorious pranks
h
Loop number: 0 Fetching char w
index for char 53 adding to X...
Loop number: 1 Fetching char h
index for char 38 adding to X...
Loop number: 2 Fetching char e
index for char 35 adding to X...
Loop number: 3 Fetching char n
index for char 44 adding to X...
Loop number: 4 Fetching char  
index for char 2 adding to X...
Loop number: 5 Fetching char e
index for char 35 adding to X...
Loop number: 6 Fetching char u
index for char 51 adding to X...
Loop number: 7 Fetching char l
index for char 42 adding to X...
Loop number: 8 Fetching char e
index for char 35 adding to X...
Loop number: 9 Fetching char n
index for char 44 adding to X...
Loop number: 10 Fetching char s
index for char 49 adding to X...
Loop number: 11 Fetching char p
index for char 46 adding to X...
Loop number: 12 Fetching char i
index for char 39 adding to X...
Loop number: 13 Fetching char e
index for char 35 adding to X...

In [17]:
X = np.array(X, dtype=np.float32)  # Make sure X is a NumPy array with the correct dtype
y = np.array(y, dtype=np.float32)  # Make sure y is a NumPy array with the correct dtype
print(nb_chars)

72


In [18]:
print(f" X type : {type(X)}")
print(f" y type: {type(y)}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print("X dtype:", X.dtype)
print("y dtype:", y.dtype)
print("NaNs in X:", np.isnan(X).sum())
print("NaNs in y:", np.isnan(y).sum())
print("Infs in X:", np.isinf(X).sum())
print("Infs in y:", np.isinf(y).sum())


 X type : <class 'numpy.ndarray'>
 y type: <class 'numpy.ndarray'>
X shape: (2, 70, 72)
y shape: (2, 72)
X dtype: float32
y dtype: float32
NaNs in X: 0
NaNs in y: 0
Infs in X: 0
Infs in y: 0


In [19]:
# Model running parametersd
model_batch_size = 73
number_of_iterations = 10
number_of_epochs_per_iteration = 1
number_of_predictions_per_epoc = 50
model_hidden_size = 128


In [103]:
def test_model(ml):
    try:
        ml(X[:2])
        print("Model input shape is compatible.")
    except Exception as e:
        print("Error with model input shape:", e)

In [104]:
# RNN model

rnnmodel = Sequential()
rnnmodel.add(SimpleRNN(model_hidden_size, 
                    return_sequences=False, 
                    input_shape=(sequence_of_chars, nb_chars), 
                    unroll=True)
         )
rnnmodel.add(Dense(nb_chars))
rnnmodel.add(Activation("softmax"))
rnnmodel.compile(loss="categorical_crossentropy", optimizer="rmsprop")
test_model(rnnmodel)

Model input shape is compatible.


In [105]:
# lstm MODEL

# Build the LSTM model
lstmmodel = Sequential()
lstmmodel.add(LSTM(model_hidden_size, 
                   return_sequences=False, 
                   input_shape=(sequence_of_chars, nb_chars), 
                   unroll=True)
             )
lstmmodel.add(Dense(nb_chars))
lstmmodel.add(Activation("softmax"))
lstmmodel.compile(loss="categorical_crossentropy", optimizer="rmsprop")
test_model(lstmmodel)

Model input shape is compatible.


In [106]:
# Model running function
def run_model(model_to_run):
    generated_texts = []
    print(f'running model {model_to_run.summary()}')
    for iteration in range(number_of_iterations):
        
        print("=" * 50)
        print(f"Iteration: {iteration}")
        model_to_run.fit(X, y, batch_size=model_batch_size, epochs=number_of_epochs_per_iteration)
        # Select random text seed to generate text from
        test_idx = np.random.randint(len(input_chars))
        test_chars = input_chars[test_idx]
        print(f"\nGenerating from seed: {test_chars}")
        print(test_chars, end="")
        generated_texts.append(test_chars)
        # Predictions based off number of predictions per epoch
        for i in range(number_of_predictions_per_epoc):
            Xtest = np.zeros((1, sequence_of_chars, nb_chars))
            for i, ch in enumerate(test_chars):
                Xtest[0, i, char2index[ch]] = 1
            # Actual predictions off model
            pred = model_to_run.predict(Xtest, verbose=0)[0]
            ypred = index2char[np.argmax(pred)]
            print(ypred, end="")
            # Move forward with test_chars + ypred
            test_chars = test_chars[1:] + ypred
    return generated_texts
print('\n')

In [107]:
lstm_generated_texts = run_model(lstmmodel)
print(f"\ngenerated text: {lstm_generated_texts}")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │       103,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 73)             │         9,417 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 73)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 112,841 (440.79 KB)

 Trainable params: 112,841 (440.79 KB)

 Non-trainable params: 0 (0.00 B)

running model None
Iteration: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 4.2797

Generating from seed: en eulenspiegel came to magdeburg—the fame of his notorious pranks
ha
en eulenspiegel came to magdeburg—the fame of his notorious pranks
havvvvvv5hrrarrbarrbarrbararrbararrbararrbararrbarar==================================================
Iteration: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 4.1969

Generating from seed: hen eulenspiegel came to magdeburg—the fame of his notorious pranks
h
hen eulenspiegel came to magdeburg—the fame of his notorious pranks
hhvvvvvvvavvvarvarvarararararararararavarararararav==================================================
Iteration: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 4.0977

Generating from seed: hen eulenspiegel came to magdeburg—the fame of his notorious pranks
h
hen eulenspiegel came to magdeburg—the fame of his notorious pranks
havvavvavvavavvhavvavvhavvavvhavvhavvhhavvvvavvavav===============================================

In [108]:
rnn_generated_texts = run_model(rnnmodel)
print(f"\ngenerated text: {rnn_generated_texts}")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 73)             │         9,417 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 73)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,273 (137.79 KB)

 Trainable params: 35,273 (137.79 KB)

 Non-trainable params: 0 (0.00 B)

running model None
Iteration: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 4.0717

Generating from seed: when eulenspiegel came to magdeburg—the fame of his notorious pranks

when eulenspiegel came to magdeburg—the fame of his notorious pranks
!7v2[y1r#p“6àe_a.[1nà)/”‘:eh•ä(-y6%/00e0)1==================================================
Iteration: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.5331

Generating from seed: en eulenspiegel came to magdeburg—the fame of his notorious pranks
ha
en eulenspiegel came to magdeburg—the fame of his notorious pranks
hav4]à1r#py6àe_avqav2]aveav1havq,veüav3,vsü,1éaveüzd==================================================
Iteration: 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.6891

Generating from seed: hen eulenspiegel came to magdeburg—the fame of his notorious pranks
h
hen eulenspiegel came to magdeburg—the fame of his notorious pranks
hav4]vvhavhav4avvhavh”vv‘av4]vhhav7av1avvhavhavh”,v==================================================
Iter

In [116]:
def get_ngrams(txt, n):
    ngrams = zip(*[txt[i:] for i in range(n)])
    return Counter(["-".join(ngram) for ngram in ngrams])

In [117]:
unigrams = get_ngrams(text_tokenized, 1)
bigrams = get_ngrams(text_tokenized, 2)
trigrams = get_ngrams(text_tokenized, 3)
print(trigrams)

Counter({':-—-“': 601, '?-”-“': 141, ')-:-—': 137, 'don-’-t': 87, '“-well-,': 80, ',-”-said': 79, 'said-:-“': 64, '“-oh-,': 61, '!-”-“': 61, '—-“-i': 56, '”-said-the': 47, '—-“-what': 41, 'i-’-ll': 37, '”-“-oh': 36, '“-i-am': 36, 'one-of-the': 34, 'professor-:-—': 34, '—-“-well': 33, '“-yes-,': 32, 'can-’-t': 32, '!-”-a': 32, ',-”-replied': 32, ':-“-i': 31, '”-“-i': 31, '—-“-oh': 31, '!-”-the': 31, '“-why-,': 30, '—-“-you': 30, ',-sir-,': 29, 'i-don-’': 28, '’-t-you': 28, '—-“-why': 26, '”-“-well': 26, '“-no-,': 24, ',-“-i': 24, ',-who-was': 24, '.-“-i': 24, ',-and-the': 23, '?-”-the': 23, ',-and-i': 21, 'student-:-—': 21, ',-“-you': 20, 'and-said-:': 20, 'what-do-you': 20, '”-“-why': 20, '?-”-asked': 20, 'the-crown-prince': 20, ',-but-the': 19, '”-replied-the': 19, 'that-’-s': 19, 'didn-’-t': 19, '“-you-are': 18, '.-“-oh': 18, 'it-’-s': 18, 'a-:-—': 18, ',-”-was': 18, ',-my-dear': 18, 'project-gutenberg™-electronic': 18, 'saying-:-“': 17, '“-what-is': 17, 'doctor-:-—': 17, '”-“-what':